Corrientes en chorro en 200hPa

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import imageio
from matplotlib.colors import LinearSegmentedColormap

ds = xr.open_dataset("ERA5_SINOP3.nc")
# Configuración de nivel de presión
nivel = 200 

# Cálculo de la velocidad del viento
u = ds['u'].sel(pressure_level=nivel)
v = ds['v'].sel(pressure_level=nivel)
velocidad = np.sqrt(u**2 + v**2)

# Colormap
colores = ['#4169E1', '#40E0D0', '#90EE90', '#FFFF99', '#FFB366', '#FF6347', '#CC0000']
cmap = LinearSegmentedColormap.from_list('viento', colores)

# Generar imágenes (15-31 de marzo de 2017)
imagenes = []
for i, tiempo in enumerate(ds.valid_time):
    
    fig = plt.figure(figsize=(12, 10))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([-120, -30, -65, 15])
    
    # Fondo
    ax.add_feature(cfeature.LAND, facecolor='white')
    ax.add_feature(cfeature.OCEAN, facecolor='white')
    ax.coastlines(linewidth=1.5, color='black')
    ax.add_feature(cfeature.BORDERS, linewidth=1, color='black')
    
    # Grillas
    gl = ax.gridlines(draw_labels=True, linewidth=0.8, alpha=0.6, 
                      linestyle='--', color='gray')
    gl.top_labels = False
    gl.right_labels = False
    
    # Ploteo de velocidad
    vel = velocidad.sel(valid_time=tiempo)
    u_t = u.sel(valid_time=tiempo)
    v_t = v.sel(valid_time=tiempo)
    
    # Barbas de viento
    paso = 3
    lons = vel.longitude[::paso].values
    lats = vel.latitude[::paso].values
    u_sub = u_t[::paso, ::paso].values
    v_sub = v_t[::paso, ::paso].values
    
    velocidad_sub = vel[::paso, ::paso].values
    Q = ax.quiver(lons, lats, u_sub, v_sub, velocidad_sub,
                  transform=ccrs.PlateCarree(),
                  cmap=cmap, scale=250, scale_units='inches',
                  width=0.002, headwidth=3, headlength=4,
                  alpha=0.85, clim=[0, 80])
    
    # Barra 
    cbar = plt.colorbar(Q, ax=ax, orientation='horizontal', 
                        pad=0.05, shrink=0.8, aspect=40)
    cbar.set_label('Intensidad del viento (m/s)', fontsize=13, fontweight='bold')
    cbar.ax.tick_params(labelsize=11)
    
    # Título
    fecha_str = str(tiempo.values)[:16]
    plt.title(f'Corrientes en chorro {fecha_str} UTC\nViento en {nivel} hPa', 
              fontsize=14, fontweight='bold', pad=20)
    
    # Guardar cada imagen generada
    plt.savefig(f'temp_{i}.png', bbox_inches='tight', dpi=120, facecolor='white')
    imagenes.append(imageio.imread(f'temp_{i}.png'))
    plt.close()
    print(f'Imagen {i+1}/{len(ds.valid_time)} - {fecha_str}')


# Creación de GIF
imageio.mimsave(f'jet_{nivel}hPa.gif', imagenes, duration=0.4, loop=0)
print(f'\n GIF creado: Jet_{nivel}hPa.gif con {len(imagenes)} frames')

Advección de temperatura en 850hPa

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy.ndimage import gaussian_filter
import cartopy.mpl.ticker as cticker
import imageio

ds = xr.open_dataset("ERA5_SINOP3.nc")
# Configuración de nivel y coordenadas
nivel = 850
lat_min, lat_max = -60, 15
lon_min, lon_max = 240, 330 

# Selección de variables
t = ds["t"].sel(pressure_level=nivel) - 273.15
u = ds["u"].sel(pressure_level=nivel)
v = ds["v"].sel(pressure_level=nivel)
z = ds["z"].sel(pressure_level=nivel)

#Generar de imágenes imágenes (15-31 de marzo de 2017)
imagenes = []

for i, tiempo in enumerate(ds.valid_time):
    t_t = t.sel(valid_time=tiempo, 
                latitude=slice(15, -60),
                longitude=slice(240, 330))
    u_t = u.sel(valid_time=tiempo,
                latitude=slice(15, -60),
                longitude=slice(240, 330))
    v_t = v.sel(valid_time=tiempo,
                latitude=slice(15, -60),
                longitude=slice(240, 330))
    z_t = z.sel(valid_time=tiempo,
                latitude=slice(15, -60),
                longitude=slice(240, 330))
    

    # Cálculo de advección
    dT_dy, dT_dx = np.gradient(t_t.values)
    
    dlat = np.abs(np.diff(t_t.latitude.values).mean())
    dlon = np.abs(np.diff(t_t.longitude.values).mean())
    
    dy = dlat * 111000  
    dx = dlon * 111000 * np.cos(np.radians(t_t.latitude.values.mean()))
    dT_dx = dT_dx / dx
    dT_dy = dT_dy / dy
    
    adv = -(u_t.values * dT_dx + v_t.values * dT_dy)
    adv = gaussian_filter(adv, sigma=1.2)
    adv_plot = adv * 1e5  # x10^-5
    
    # Convertir longitudes a -180/180 para visualización
    lon_plot = t_t.longitude.values
    lon_plot = np.where(lon_plot > 180, lon_plot - 360, lon_plot)
    
    # Ploteo
    fig = plt.figure(figsize=(9, 10))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([-120, -30, -60, 15]) 
    
    # Mapa base
    ax.add_feature(cfeature.COASTLINE, linewidth=0.7)
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.add_feature(cfeature.LAND, facecolor="white")
    
    gl = ax.gridlines(draw_labels=True, linewidth=0.4, color='gray', 
                      alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {"size": 9}
    gl.ylabel_style = {"size": 9}
    gl.xformatter = cticker.LongitudeFormatter()
    gl.yformatter = cticker.LatitudeFormatter()
    
    # Advección (shade)
    levels = np.linspace(-20, 20, 21)
    cf = ax.contourf(
        lon_plot, t_t.latitude, adv_plot,
        levels=levels, cmap="RdBu_r", extend="both"
    )
    
    #Contornos de geopotencial
    hgt = z_t / 9.81
    cs = ax.contour(
        lon_plot, t_t.latitude, hgt,
        levels=np.arange(1300, 1600, 20),
        colors="black", linewidths=0.6
    )
    ax.clabel(cs, fmt="%d", fontsize=7)
    
    # Vectores de viento
    skip = (slice(None, None, 5), slice(None, None, 5))
    ax.quiver(
        lon_plot[skip[1]], t_t.latitude[skip[0]],
        u_t.values[skip], v_t.values[skip],
        scale=700, width=0.0018, headwidth=3, alpha=0.9
    )
    
    # Barra de color 
    cbar = plt.colorbar(cf, ax=ax, orientation="vertical", 
                        pad=0.02, aspect=35, shrink=0.85)
    cbar.set_label("Advección de Temperatura (×10⁻⁵ °C s⁻¹)")
    
    # Título 
    fecha_str = str(tiempo.values)[:16]
    ax.set_title(f"Advección de Temperatura ({nivel} hPa)\n{fecha_str} UTC",
                 fontsize=12, weight="bold", pad=10)
    
    plt.tight_layout()
    
    # Guardar imagen
    nombre_img = f'adv_temp_{i:04d}.png'
    plt.savefig(nombre_img, dpi=100, bbox_inches='tight', facecolor='white')
    imagenes.append(imageio.imread(nombre_img))
    plt.close()
    
    print(f' {i+1}/{len(ds.valid_time)} - {fecha_str}')

# === 11. Crear GIF ===
imageio.mimsave(f'adveccion_temp_{nivel}hPa.gif', imagenes, duration=0.4, loop=0)
print(f'\n GIF creado: adveccion_temp_{nivel}hPa.gif con {len(imagenes)} frames')